In [1161]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import re
from os import path
from glob import glob
from tqdm import tqdm
import numpy as np
from matplotlib import cm
from pprint import pprint

In [1162]:
ROOT_PATH = '\\\\CILab_NAS_Data/homes/inchang/PuzzlePCG/ProxyPlayer'

In [1163]:
files = glob(path.join(ROOT_PATH, '*', 'MatchResult_*.csv'))


In [1164]:
StatisticsColumns = [
    'ReachedKnowledgeStep',
    #'AlmostReachedKnowledgeStep',
    'Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece']

In [1165]:
replacements = {
    'met': 'method',
    'pla': 'player',
    'eff': 'simpleEffect',
    'obj': 'objective',
    'sim': 'simulation',
    'rat': 'almostRatio',
    'pdep': 'playerDepth',
}

def parse_filename(data):
    result = dict()
    for i in range(0, len(data), 2):
        if data[i] in replacements.keys():
            data[i] = replacements[data[i]]

        result[data[i]] = data[i+1]

    return result

In [1166]:

dfs = []
for file_path in tqdm(files):
    # print(file_path)
    file_name = path.basename(file_path)

    _file_name = file_name.split('_')[1:-1]
    args = parse_filename(_file_name)

    df = pd.read_csv(file_path)

    for key, value in args.items():
        df[key] = value
    dfs.append(df)

original_df = pd.concat(dfs)

original_df['Sum_Matched'] = original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece']  + \
    original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece']
original_df.loc[original_df['method'] == 'random', 'simulation'] = 0

original_df['simpleEffect'] = original_df['simpleEffect'].replace(np.nan, 0)
original_df['objective'] = original_df['objective'].replace(np.nan, '-')

original_df.loc[original_df['method'] == 'random', 'playerDepth'] = '-'
original_df.loc[original_df['method'] == 'random', 'simulation'] = '-'
original_df.loc[original_df['method'] == 'random', 'almostRatio'] = '-'
original_df.loc[original_df['objective'] == 'kp', 'almostRatio'] = '-'
original_df.loc[original_df['objective'] == 'score', 'almostRatio'] = '-'

original_df.to_csv('original_df.csv')


100%|██████████| 88/88 [00:01<00:00, 68.27it/s]


In [1167]:
# original_df = pd.read_csv('original_df.csv')
print(set(original_df['objective']))

{'-', 'knowledge', 'score'}


In [1168]:
set(original_df['playerDepth'])


{'-', '1', '2'}

In [1169]:
original_df.head()


,EpisodeCount,StepCount,Time,InstanceUUID,SettleCount,ChangedCount,MeanComparisonCount,StdComparisonCount,ReachedKnowledgeStep,Q1ReachedKnowledgeStep,...,Target_BombPiece,Target_RainbowPiece,method,player,simpleEffect,objective,simulation,almostRatio,playerDepth,Sum_Matched
0,1,200,2023-05-24 13:00:19.842,1b9ad715,0,229,0.914557,0.700777,41,40,...,2,3,mcts,0,1,knowledge,200,1.0,1,330
1,2,200,2023-05-24 13:01:57.884,1b9ad715,0,250,0.965625,0.644781,47,37,...,2,3,mcts,0,1,knowledge,200,1.0,1,396
2,3,200,2023-05-24 13:04:01.445,1b9ad715,0,269,0.994302,0.700996,55,44,...,2,3,mcts,0,1,knowledge,200,1.0,1,600
3,4,200,2023-05-24 13:06:00.311,1b9ad715,0,269,0.994269,0.694785,36,11,...,2,3,mcts,0,1,knowledge,200,1.0,1,468
4,5,200,2023-05-24 13:07:52.564,1b9ad715,0,255,0.978723,0.682649,73,48,...,2,3,mcts,0,1,knowledge,200,1.0,1,480


In [1170]:
original_df.columns.values.tolist()

['EpisodeCount',
 'StepCount',
 'Time',
 'InstanceUUID',
 'SettleCount',
 'ChangedCount',
 'MeanComparisonCount',
 'StdComparisonCount',
 'ReachedKnowledgeStep',
 'Q1ReachedKnowledgeStep',
 'Q2ReachedKnowledgeStep',
 'Q3ReachedKnowledgeStep',
 'Matched_HorizontalPiece',
 'Matched_VerticalPiece',
 'Matched_CrossPiece',
 'Matched_RocketPiece',
 'Matched_BombPiece',
 'Matched_RainbowPiece',
 'Target_HorizontalPiece',
 'Target_VerticalPiece',
 'Target_CrossPiece',
 'Target_RocketPiece',
 'Target_BombPiece',
 'Target_RainbowPiece',
 'method',
 'player',
 'simpleEffect',
 'objective',
 'simulation',
 'almostRatio',
 'playerDepth',
 'Sum_Matched']

In [1171]:
pd.set_option('display.max_rows', None)
match_result = original_df.groupby(['player', 'method', 'objective', 'playerDepth', 'almostRatio', 'simulation'])[
                                      'method',
                                      'Matched_HorizontalPiece',
                                      'Matched_VerticalPiece',
                                      'Matched_CrossPiece',
                                      'Matched_RocketPiece',
                                      'Matched_BombPiece',
                                      'Matched_RainbowPiece',
                                      #'AlmostReachedKnowledgeStep',
                                      'Q1ReachedKnowledgeStep',
                                      'Q2ReachedKnowledgeStep',
                                      'Q3ReachedKnowledgeStep',
                                      'ReachedKnowledgeStep',
                                      'Sum_Matched'].agg(
    {
        'method': ['count'],
        #'AlmostReachedKnowledgeStep': ['mean'],
        'Q1ReachedKnowledgeStep': ['mean'],
        'Q2ReachedKnowledgeStep': ['mean'],
        'Q3ReachedKnowledgeStep': ['mean'],
        'ReachedKnowledgeStep': ['mean'],
        'Matched_HorizontalPiece':['mean'],
        'Matched_VerticalPiece':['mean'],
        'Matched_CrossPiece':['mean'],
        'Matched_RocketPiece':['mean'],
        'Matched_BombPiece':['mean'],
        'Matched_RainbowPiece':['mean'],
        'Sum_Matched':['mean'],
     }
)
match_result


C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


method  \
                                                            count   
player method objective playerDepth almostRatio simulation          
0      mcts   knowledge 1           1.0         200           210   
                        2           1.0         200           115   
              score     1           -           200           142   
       random -         -           -           -             700   
1      mcts   knowledge 1           1.0         200           210   
                        2           1.0         200           112   
              score     1           -           200           140   
       random -         -           -           -             695   
10     mcts   knowledge 1           1.0         200           211   
                        2           1.0         200           110   
              score     1           -           200           100   
       random -         -           -           -             673   
2      mcts   knowledge 1           1.0         200           206   
                        2           1.0         200           115   
              score     1           -           200           138   
       random -         -           -           -             694   
3      mcts   knowledge 1           1.0         200           207   
                        2           1.0         200           114   
              score     1           -           200           141   
       random -         -           -           -             682   
4      mcts   knowledge 1           1.0         200           207   
                        2           1.0         200           112   
              score     1           -           200           142   
       random -         -           -           -             691   
5      mcts   knowledge 1           1.0         200           210   
                        2           1.0         200           111   
              score     1           -           200           120   
       random -         -           -           -             690   
6      mcts   knowledge 1           1.0         200           206   
                        2           1.0         200           115   
              score     1           -           200           109   
       random -         -           -           -             678   
7      mcts   knowledge 1           1.0         200           205   
                        2           1.0         200           110   
              score     1           -           200           102   
       random -         -           -           -             685   
8      mcts   knowledge 1           1.0         200           204   
                        2           1.0         200           113   
              score     1           -           200           100   
       random -         -           -           -             676   
9      mcts   knowledge 1           1.0         200           206   
                        2           1.0         200           114   
              score     1           -           200           104   
       random -         -           -           -             691   

                                                           Q1ReachedKnowledgeStep  \
                                                                             mean   
player method objective playerDepth almostRatio simulation                          
0      mcts   knowledge 1           1.0         200                     36.614286   
                        2           1.0         200                     40.721739   
              score     1           -           200                     34.401408   
       random -         -           -           -                       59.281429   
1      mcts   knowledge 1           1.0         200                     42.023810   
                        2           1.0         200                     45.955357   
              score     1           -           20

In [1172]:
original_df = original_df[
    (original_df['ReachedKnowledgeStep'] != -1) &
    (original_df['Q1ReachedKnowledgeStep'] != -1) &
    (original_df['Q2ReachedKnowledgeStep'] != -1) &
    (original_df['Q3ReachedKnowledgeStep'] != -1)
]

for i in range(5):
    original_df['playerDepth'] = original_df['playerDepth'].replace(str(i), i)
for i in range(100, 1000, 100):
    original_df['simulation'] = original_df['simulation'].replace(str(i), i)

In [1173]:
pd.set_option('display.max_rows', None)
match_result = original_df.groupby(['player', 'method', 'objective', 'playerDepth', 'almostRatio', 'simulation'])[
                                      'method',
                                      'Matched_HorizontalPiece',
                                      'Matched_VerticalPiece',
                                      'Matched_CrossPiece',
                                      'Matched_RocketPiece',
                                      'Matched_BombPiece',
                                      'Matched_RainbowPiece',
                                      #'AlmostReachedKnowledgeStep',
                                      'Q1ReachedKnowledgeStep',
                                      'Q2ReachedKnowledgeStep',
                                      'Q3ReachedKnowledgeStep',
                                      'ReachedKnowledgeStep',
                                      'Sum_Matched'].agg(
    {
        'method': ['count'],
        #'AlmostReachedKnowledgeStep': ['mean'],
        'Q1ReachedKnowledgeStep': ['mean'],
        'Q2ReachedKnowledgeStep': ['mean'],
        'Q3ReachedKnowledgeStep': ['mean'],
        'ReachedKnowledgeStep': ['mean'],
        'Matched_HorizontalPiece':['mean'],
        'Matched_VerticalPiece':['mean'],
        'Matched_CrossPiece':['mean'],
        'Matched_RocketPiece':['mean'],
        'Matched_BombPiece':['mean'],
        'Matched_RainbowPiece':['mean'],
        'Sum_Matched':['mean'],
     }
)
match_result


C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


method  \
                                                            count   
player method objective playerDepth almostRatio simulation          
0      mcts   knowledge 1           1.0         200           208   
                        2           1.0         200           114   
              score     1           -           200           141   
       random -         -           -           -             630   
1      mcts   knowledge 1           1.0         200           207   
                        2           1.0         200           109   
              score     1           -           200           137   
       random -         -           -           -             524   
10     mcts   knowledge 1           1.0         200           211   
                        2           1.0         200           110   
              score     1           -           200           100   
       random -         -           -           -             610   
2      mcts   knowledge 1           1.0         200           195   
                        2           1.0         200           111   
              score     1           -           200           135   
       random -         -           -           -             565   
3      mcts   knowledge 1           1.0         200           205   
                        2           1.0         200           109   
              score     1           -           200           139   
       random -         -           -           -             543   
4      mcts   knowledge 1           1.0         200           206   
                        2           1.0         200           112   
              score     1           -           200           142   
       random -         -           -           -             611   
5      mcts   knowledge 1           1.0         200           207   
                        2           1.0         200           110   
              score     1           -           200           118   
       random -         -           -           -             626   
6      mcts   knowledge 1           1.0         200           206   
                        2           1.0         200           115   
              score     1           -           200           107   
       random -         -           -           -             591   
7      mcts   knowledge 1           1.0         200           201   
                        2           1.0         200           109   
              score     1           -           200           102   
       random -         -           -           -             610   
8      mcts   knowledge 1           1.0         200           204   
                        2           1.0         200           113   
              score     1           -           200           100   
       random -         -           -           -             639   
9      mcts   knowledge 1           1.0         200           185   
                        2           1.0         200           102   
              score     1           -           200            93   
       random -         -           -           -             339   

                                                           Q1ReachedKnowledgeStep  \
                                                                             mean   
player method objective playerDepth almostRatio simulation                          
0      mcts   knowledge 1           1.0         200                     36.057692   
                        2           1.0         200                     40.903509   
              score     1           -           200                     34.304965   
       random -         -           -           -                       57.671429   
1      mcts   knowledge 1           1.0         200                     41.347826   
                        2           1.0         200                     43.440367   
              score     1           -           20

In [1174]:
pd.set_option('display.max_rows', None)


match_result = original_df.copy()
match_result = match_result[~match_result['almostRatio'].isin(['0.75'])]
# match_result = match_result[~match_result['playerDepth'].isin([2])]
match_result = match_result[~match_result['simulation'].isin([100, 400])]

# print(set(match_result['playerDepth']))
pd.set_option('display.max_rows', None)
match_result = match_result.groupby(['method', 'simpleEffect','objective', 'playerDepth', 'simulation'])[
                                      'method',
                                      'Matched_HorizontalPiece',
                                      'Matched_VerticalPiece',
                                      'Matched_CrossPiece',
                                      'Matched_RocketPiece',
                                      'Matched_BombPiece',
                                      'Matched_RainbowPiece',
                                      #'AlmostReachedKnowledgeStep',
                                      'Q1ReachedKnowledgeStep',
                                      'Q2ReachedKnowledgeStep',
                                      'Q3ReachedKnowledgeStep',
                                      'ReachedKnowledgeStep',
                                      'Sum_Matched'].agg(
    {
        'method': ['count'],
        #'AlmostReachedKnowledgeStep': ['mean'],
        'Q1ReachedKnowledgeStep': ['mean'],
        'Q2ReachedKnowledgeStep': ['mean'],
        'Q3ReachedKnowledgeStep': ['mean'],
        'ReachedKnowledgeStep': ['mean'],
        'Matched_HorizontalPiece':['mean'],
        'Matched_VerticalPiece':['mean'],
        'Matched_CrossPiece':['mean'],
        'Matched_RocketPiece':['mean'],
        'Matched_BombPiece':['mean'],
        'Matched_RainbowPiece':['mean'],
        'Sum_Matched':['mean'],
     }
)
match_result


C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if sys.path[0] == "":


method  \
                                                      count   
method simpleEffect objective playerDepth simulation          
mcts   0            knowledge 1           200          1123   
                              2           200           608   
                    score     1           200           665   
       1            knowledge 1           200          1112   
                              2           200           606   
                    score     1           200           649   
random 0            -         -           -            3182   
       1            -         -           -            3106   

                                                     Q1ReachedKnowledgeStep  \
                                                                       mean   
method simpleEffect objective playerDepth simulation                          
mcts   0            knowledge 1           200                     41.550312   
                              2           200                     43.090461   
                    score     1           200                     41.288722   
       1            knowledge 1           200                     43.916367   
                              2           200                     43.107261   
                    score     1           200                     42.574730   
random 0            -         -           -                       65.549026   
       1            -         -           -                       68.959755   

                                                     Q2ReachedKnowledgeStep  \
                                                                       mean   
method simpleEffect objective playerDepth simulation                          
mcts   0            knowledge 1           200                     50.622440   
                              2           200                     51.194079   
                    score     1           200                     50.654135   
       1            knowledge 1           200                     53.205935   
                              2           200                     51.351485   
                    score     1           200                     52.616333   
random 0            -         -           -                       76.338781   
       1            -         -           -                       79.723760   

                                                     Q3ReachedKnowledgeStep  \
                                                                       mean   
method simpleEffect objective playerDepth simulation                          
mcts   0            knowledge 1           200                     65.239537   
                              2           200                     65.253289   
                    score     1           200                     64.718797   
       1            knowledge 1           200                     69.210432   
                              2           200                     66.694719   
                    score     1           200                     69.130971   
random 0            -         -           -                       95.824953   
       1            -         -           -                       99.744688   

                                                     ReachedKnowledgeStep  \
                                                                     mean   
method simpleEffect objective playerDepth simulation                        
mcts   0            knowledge 1           200                   70.484417   
                              2           200                   70.815789   
                    score     1           200                   70.479699   
       1            knowledge 1           200                   74.544065   
                              2           200                   72.488449   
                    score     1           200                   74.006163   
random 0            -         -           -             

In [1175]:
def get_legend_name(x):
    if x['method'] == 'mcts':
        if x['objective'] == 'score':
            return f"{x['method']}_{x['objective']}_{x['simulation']}_{x['playerDepth']}"
        elif x['objective'] == 'knowledge':
            return f"{x['method']}_{x['objective']}_{x['almostRatio']}_{x['simulation']}_{x['playerDepth']}"
        elif x['objective'] == 'kp':
            return f"{x['method']}_{x['objective']}_{x['almostRatio']}_{x['simulation']}_{x['playerDepth']}"
    else:
        return f"{x['method']}"

In [1176]:
melted_df = original_df.copy()

melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation', 'player']).sample(n=100, random_state=1)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
pprint(set(melted_df['concat_method']))
# Display Target
melted_df = melted_df[melted_df['concat_method'].isin(['random',
                                                       'mcts_knowledge_1.0_400_1',
                                                       'mcts_knowledge_0.75_400_1',
                                                       'mcts_knowledge_1.0_400_2',
                                                       'mcts_knowledge_0.75_400_2',
                                                       'mcts_score_400_2'])]

plt.figure(figsize=(7, 5))
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*StatisticsColumns], ignore_index=False)

melted_df['x'] = melted_df['variable']
# melted_df['x'] = melted_df['x'].str.replace('AlmostReachedKnowledgeStep', 'ar_step')	125.0
melted_df['x'] = melted_df['x'].str.replace('ReachedKnowledgeStep', 'r_step')

melted_df['x'] = melted_df['x'].str.replace('Matched_HorizontalPiece', 'b0')
melted_df['x'] = melted_df['x'].str.replace('Matched_VerticalPiece', 'b1')
melted_df['x'] = melted_df['x'].str.replace('Matched_CrossPiece', 'b2')
melted_df['x'] = melted_df['x'].str.replace('Matched_RocketPiece', 'b3')
melted_df['x'] = melted_df['x'].str.replace('Matched_BombPiece', 'b4')
melted_df['x'] = melted_df['x'].str.replace('Matched_RainbowPiece', 'b5')

ax = sns.boxplot(x="x", y="value", hue="player", data=melted_df)
plt.xticks(rotation=45)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
for player_no in range(0, 4):
    player_df = original_df[original_df['player'] == player_no]
    melted_df = player_df.copy()

    melted_df.rename(columns={
        'ReachedKnowledgeStep':'100',
        'Q1ReachedKnowledgeStep':'25',
        'Q2ReachedKnowledgeStep':'50',
        'Q3ReachedKnowledgeStep':'75'
    }, inplace=True)

    _columns = [
    '25', '50', '75', '100',
    # Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece'
    ]

    # print(player_df.keys())
    sns.set_palette('Set2')
    melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=400, random_state=1, replace=True)
    melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
    melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                        value_vars=[*_columns], ignore_index=False)
    # pprint(set(melted_df['concat_method']))
    _methods = [
        'mcts_kp_-_400_2',
        'mcts_knowledge_1.0_400_2',
        'mcts_score_400_2',
        'random'
    ]
    melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

    plt.figure(figsize=(5, 4))
    plt.clf()
    ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

    plt.legend(loc='upper left')
    ax.set_title(f'Required Steps for Learning Skill (Player: {player_no})')
    ax.set_xlabel('Progress (%)')
    ax.set_ylabel('Step')
    plt.savefig(f'./figures/learning_steps_p{player_no}', dpi=300)
    plt.show()

In [ ]:
player_df = original_df.copy()
melted_df = player_df.copy()

melted_df.rename(columns={
    'ReachedKnowledgeStep':'100',
    'Q1ReachedKnowledgeStep':'25',
    'Q2ReachedKnowledgeStep':'50',
    'Q3ReachedKnowledgeStep':'75'
}, inplace=True)

_columns = [
'25', '50', '75', '100',
# Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece'
]

# print(player_df.keys())
sns.set_palette('Set2')
melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=400, random_state=1, replace=True)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*_columns], ignore_index=False)
# pprint(set(melted_df['concat_method']))
_methods = [
    'mcts_kp_-_400_2',
    'mcts_knowledge_1.0_400_2',
    'mcts_score_400_2',
    'random'
]
melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

plt.figure(figsize=(5, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

plt.legend(loc='upper left')
ax.set_title(f'Required Steps for Learning Skill (Player: All)')
ax.set_xlabel('Progress (%)')
ax.set_ylabel('Step')
plt.savefig(f'./figures/learning_steps_pA', dpi=300)
plt.show()



In [ ]:
for player_no in range(0, 4):
    player_df = original_df[original_df['player'] == player_no]
    melted_df = player_df.copy()

    melted_df.rename(columns={
        'ReachedKnowledgeStep':'100',
        'Q1ReachedKnowledgeStep':'25',
        'Q2ReachedKnowledgeStep':'50',
        'Q3ReachedKnowledgeStep':'75',
        'Matched_HorizontalPiece':'Horizon',
        'Matched_VerticalPiece':'Vertical',
        'Matched_CrossPiece':'Cross',
        'Matched_RocketPiece':'Rocket',
        'Matched_BombPiece':'Bomb',
        'Matched_RainbowPiece':'Rainbow',

    }, inplace=True)

    _columns = [
    # '25', '50', '75', '100',
    'Horizon', 'Vertical', 'Cross', 'Rocket', 'Bomb', 'Rainbow'
    ]

    # print(player_df.keys())
    sns.set_palette('Set3')
    melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=300, random_state=1, replace=True)
    melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
    melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                        value_vars=[*_columns], ignore_index=False)
    # pprint(set(melted_df['concat_method']))
    _methods = [
        'mcts_kp_-_400_2',
        'mcts_knowledge_1.0_400_2',
        'mcts_score_400_2',
        'random'
    ]
    melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

    plt.figure(figsize=(5, 4))
    plt.clf()
    ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, linewidth=0.8)

    plt.legend(loc='upper left')
    ax.set_title(f'Matched Blocks (Player: {player_no})')
    ax.set_xlabel('Block Type')
    ax.set_ylabel('Count')
    plt.xticks(rotation=45)
    plt.savefig(f'./figures/block_counts_p{player_no}', dpi=300)
    plt.show()

In [ ]:

player_df = original_df.copy()
melted_df = player_df.copy()

melted_df.rename(columns={
    'ReachedKnowledgeStep':'100',
    'Q1ReachedKnowledgeStep':'25',
    'Q2ReachedKnowledgeStep':'50',
    'Q3ReachedKnowledgeStep':'75',
    'Matched_HorizontalPiece':'Horizon',
    'Matched_VerticalPiece':'Vertical',
    'Matched_CrossPiece':'Cross',
    'Matched_RocketPiece':'Rocket',
    'Matched_BombPiece':'Bomb',
    'Matched_RainbowPiece':'Rainbow',

}, inplace=True)

_columns = [
# '25', '50', '75', '100',
'Horizon', 'Vertical', 'Cross', 'Rocket', 'Bomb', 'Rainbow'
]

# print(player_df.keys())
sns.set_palette('Set3')
melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=300, random_state=1, replace=True)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*_columns], ignore_index=False)
# pprint(set(melted_df['concat_method']))
_methods = [
    'mcts_kp_-_400_2',
    'mcts_knowledge_1.0_400_2',
    'mcts_score_400_2',
    'random'
]
melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

plt.figure(figsize=(5, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, linewidth=0.8)

plt.legend(loc='upper left')
ax.set_title(f'Matched Blocks (Player: All)')
ax.set_xlabel('Block Type')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.savefig(f'./figures/block_counts_pA', dpi=300)
plt.show()

In [ ]:

for player_no in range(0, 4):
    player_df = original_df[original_df['player'] == player_no]
    melted_df = player_df.copy()

    melted_df.rename(columns={
        'ReachedKnowledgeStep':'100',
        'Q1ReachedKnowledgeStep':'25',
        'Q2ReachedKnowledgeStep':'50',
        'Q3ReachedKnowledgeStep':'75'
    }, inplace=True)

    _columns = [
    '25', '50', '75', '100',
    # Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece'
    ]

    # print(player_df.keys())
    sns.set_palette('Set2')
    melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=400, random_state=1, replace=True)
    melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
    melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                        value_vars=[*_columns], ignore_index=False)
    # pprint(set(melted_df['concat_method']))
    _methods = [
        'mcts_kp_-_400_2',
        'mcts_knowledge_1.0_400_2',
        'mcts_score_400_2',
        'random'
    ]
    melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

    plt.figure(figsize=(5, 4))
    plt.clf()
    ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

    plt.legend(loc='upper left')
    ax.set_title(f'Required Steps for Learning Skill (Player: {player_no})')
    ax.set_xlabel('Progress (%)')
    ax.set_ylabel('Step')
    plt.savefig(f'./figures/learning_steps_p{player_no}', dpi=300)
    plt.show()

In [ ]:

player_df = original_df.copy()
melted_df = player_df.copy()

melted_df.rename(columns={
    'ReachedKnowledgeStep':'100',
    'Q1ReachedKnowledgeStep':'25',
    'Q2ReachedKnowledgeStep':'50',
    'Q3ReachedKnowledgeStep':'75',
    'Matched_HorizontalPiece':'Horizon',
    'Matched_VerticalPiece':'Vertical',
    'Matched_CrossPiece':'Cross',
    'Matched_RocketPiece':'Rocket',
    'Matched_BombPiece':'Bomb',
    'Matched_RainbowPiece':'Rainbow',

}, inplace=True)

_columns = [
# '25', '50', '75', '100',
'Horizon', 'Vertical', 'Cross', 'Rocket', 'Bomb', 'Rainbow'
]

# print(player_df.keys())
sns.set_palette('Set3')
melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=300, random_state=1, replace=True)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*_columns], ignore_index=False)
# pprint(set(melted_df['concat_method']))
_methods = [
    'mcts_kp_-_400_2',
    'mcts_knowledge_1.0_400_2',
    'mcts_score_400_2',
    'random'
]
melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

plt.figure(figsize=(5, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, linewidth=0.8)

plt.legend(loc='upper left')
ax.set_title(f'Matched Blocks (Player: All)')
ax.set_xlabel('Block Type')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.savefig(f'./figures/block_counts_pA', dpi=300)
plt.show()
